### 데이터 값 목록 전체

* 데이터 값 전체 자료를 csv 파일로 저장

In [2]:
import os
import pandas as pd

# 파일 저장 위치
path_dir = "D:/PythonProject/data-gatherer/dart_fs_notes/items/"
file_list = os.listdir(path_dir)

df_all = pd.DataFrame()    
for file_name in file_list:
    if file_name != 'items_all.csv' and not file_name.startswith('__'):
        df = pd.read_csv(path_dir + file_name, dtype=object)
        df_all = pd.concat([df_all, df])
    
df_all.drop_duplicates(subset=['corp_code','rpt_num'], keep='first', inplace=True, ignore_index=True)            
df_all.to_csv(path_dir + 'items_all.csv', index=False)

### 파일에 저장된 데이터 값 목록 검색

In [19]:
import pandas as pd
# 데이터 Loading
path_dir = "D:/PythonProject/data-gatherer/dart_fs_notes/items/"
df = pd.read_csv(path_dir + 'items_all.csv', dtype=object)
num_of_rpt = int(df.shape[0])
num_of_corp = len(df['corp_code'].unique())
print(f"기업 {num_of_corp}, 사업보고서 {num_of_rpt}")

기업 342, 사업보고서 2444


In [5]:
df['doc_result'].value_counts()

doc_result
OK                              2367
No Financial Statement Notes      77
Name: count, dtype: int64

In [6]:
df['unit'].value_counts()

unit
천원     1488
백만원     300
원       172
Name: count, dtype: int64

In [9]:
# 수기 보정 필요한 건
str_expr = "val_a == '-1.0' or val_b == '-1.0'"
df.query(str_expr)

,corp_code,stock_code,corp_cls,corp_name,rpt_num,rpt_name,doc_title,doc_result,unit,val_a,val_b,item_result
953,00134963,004430,Y,송원산업,20230302000104,사업보고서 (2022.12),5. 재무제표 주석,OK,백만원,65649.0,-1.0,OK
1733,00218575,032560,Y,황금에스티,20220321000623,사업보고서 (2021.12),5. 재무제표 주석,OK,천원,-1.0,0.0,OK
1735,00218575,032560,Y,황금에스티,20220225000997,[첨부정정]사업보고서 (2020.12),주석,OK,천원,-1.0,0.0,OK
1736,00218575,032560,Y,황금에스티,20170331000866,사업보고서 (2016.12),5. 재무제표 주석,OK,천원,-1.0,0.0,OK
1737,00218575,032560,Y,황금에스티,20190329004157,사업보고서 (2018.12),5. 재무제표 주석,OK,천원,-1.0,0.0,OK
1738,00218575,032560,Y,황금에스티,20180330001418,사업보고서 (2017.12),5. 재무제표 주석,OK,천원,-1.0,0.0,OK
1739,00218575,032560,Y,황금에스티,20220224003906,[기재정정]사업보고서 (2019.12),5. 재무제표 주석,OK,천원,-1.0,0.0,OK
1893,00356370,051900,Y,LG생활건강,20220516002048,[기재정정]사업보고서 (2021.12),5. 재무제표 주석,OK,백만원,-1.0,-1.0,OK
1894,00356370,051900,Y,LG생활건강,20210311001010,사업보고서 (2020.12),5. 재무제표 주석,OK,백만원,-1.0,-1.0,OK
1896,00356370,051900,Y,LG생활건강,20230320000833,사업보고서 (2022.12),5. 재무제표 주석,OK,백만원,-1.0,-1.0,OK


In [15]:
# 오류 찾기: "금액 단위 찾기 오류" ~ 재무제표 주석 문서가 있으면서 단위는 없고 금액은 있는 것
df.query("doc_result == 'OK' and unit.isna() and (val_a != '0.0' or val_b != '0.0')")

,corp_code,stock_code,corp_cls,corp_name,rpt_num,rpt_name,doc_title,doc_result,unit,val_a,val_b,item_result
130,00105271,002380,Y,케이씨씨,20230316000762,사업보고서 (2022.12),5. 재무제표 주석,OK,NaN,0.0,248065367.0,OK
131,00105271,002380,Y,케이씨씨,20220317001213,사업보고서 (2021.12),5. 재무제표 주석,OK,NaN,0.0,244518350.0,OK
132,00105271,002380,Y,케이씨씨,20210318000812,[기재정정]사업보고서 (2020.12),5. 재무제표 주석,OK,NaN,0.0,245451825.0,OK
133,00105271,002380,Y,케이씨씨,20210317000396,[기재정정]사업보고서 (2017.12),5. 재무제표 주석,OK,NaN,0.0,297653441.0,OK
134,00105271,002380,Y,케이씨씨,20210317000416,[기재정정]사업보고서 (2018.12),5. 재무제표 주석,OK,NaN,0.0,308455430.0,OK
...,...,...,...,...,...,...,...,...,...,...,...,...
2406,01319899,302440,Y,SK바이오사이언스,20230317000869,사업보고서 (2022.12),5. 재무제표 주석,OK,NaN,25621328.0,39576873.0,OK
2407,01319899,302440,Y,SK바이오사이언스,20220316001334,사업보고서 (2021.12),5. 재무제표 주석,OK,NaN,24999395.0,20753336.0,OK
2408,01319899,302440,Y,SK바이오사이언스,20201127001236,[기재정정]사업보고서 (2019.12),5. 재무제표 주석,OK,NaN,19660070.0,17644927.0,OK
2409,01319899,302440,Y,SK바이오사이언스,20190401003641,사업보고서 (2018.12),5. 재무제표 주석,OK,NaN,16072049.0,14298540.0,OK


In [16]:
# 오류 찾기: "값 찾기 오류" ~ 재무제표 주석 문서가 있으면서 val_a는 못 찾고 val_b는 찾은 경우
df.query("doc_result == 'OK' and val_a == '0.0' and val_b != '0.0'")

,corp_code,stock_code,corp_cls,corp_name,rpt_num,rpt_name,doc_title,doc_result,unit,val_a,val_b,item_result
6,00100939,000860,Y,강남제비스코,20181231000132,[기재정정]사업보고서 (2016.12),5. 재무제표 주석,OK,백만원,0.0,23547.0,OK
130,00105271,002380,Y,케이씨씨,20230316000762,사업보고서 (2022.12),5. 재무제표 주석,OK,NaN,0.0,248065367.0,OK
131,00105271,002380,Y,케이씨씨,20220317001213,사업보고서 (2021.12),5. 재무제표 주석,OK,NaN,0.0,244518350.0,OK
132,00105271,002380,Y,케이씨씨,20210318000812,[기재정정]사업보고서 (2020.12),5. 재무제표 주석,OK,NaN,0.0,245451825.0,OK
133,00105271,002380,Y,케이씨씨,20210317000396,[기재정정]사업보고서 (2017.12),5. 재무제표 주석,OK,NaN,0.0,297653441.0,OK
...,...,...,...,...,...,...,...,...,...,...,...,...
2349,01236897,271980,Y,제일약품,20230316001404,사업보고서 (2022.12),5. 재무제표 주석,OK,NaN,0.0,40761035.0,OK
2350,01236897,271980,Y,제일약품,20210315000881,사업보고서 (2020.12),5. 재무제표 주석,OK,NaN,0.0,34231394.0,OK
2351,01236897,271980,Y,제일약품,20220315001104,사업보고서 (2021.12),5. 재무제표 주석,OK,NaN,0.0,42433388.0,OK
2352,01236897,271980,Y,제일약품,20200330004024,사업보고서 (2019.12),5. 재무제표 주석,OK,NaN,0.0,32227088.0,OK


In [13]:
# 오류 찾기: 재무제표 주석 문서가 있으면서 금액과 단위가 없는 것 ~ 오류인지 아닌지 직접 확인 필요
df.query("doc_result == 'OK' and unit.isna() and val_a == '0.0' and val_b == '0.0'")

,corp_code,stock_code,corp_cls,corp_name,rpt_num,rpt_name,doc_title,doc_result,unit,val_a,val_b,item_result
41,00101664,214390,Y,경보제약,20170331001705,사업보고서 (2016.12),5. 재무제표 주석,OK,NaN,0.0,0.0,OK
45,00102113,012610,Y,경인양행,20170330000245,사업보고서 (2016.12),5. 재무제표 주석,OK,NaN,0.0,0.0,OK
46,00102113,012610,Y,경인양행,20180330000601,사업보고서 (2017.12),5. 재무제표 주석,OK,NaN,0.0,0.0,OK
47,00102113,012610,Y,경인양행,20190329002398,사업보고서 (2018.12),5. 재무제표 주석,OK,NaN,0.0,0.0,OK
48,00102113,012610,Y,경인양행,20200330000629,사업보고서 (2019.12),5. 재무제표 주석,OK,NaN,0.0,0.0,OK
...,...,...,...,...,...,...,...,...,...,...,...,...
2434,01436558,404990,Y,신한서부티엔디리츠,20220914000218,사업보고서 (2022.06),5. 재무제표 주석,OK,NaN,0.0,0.0,OK
2435,01436558,404990,Y,신한서부티엔디리츠,20230315000424,사업보고서 (2022.12),5. 재무제표 주석,OK,NaN,0.0,0.0,OK
2436,01436558,404990,Y,신한서부티엔디리츠,20230913000210,사업보고서 (2023.06),5. 재무제표 주석,OK,NaN,0.0,0.0,OK
2440,01532603,378850,Y,화승알앤에이,20220321000964,사업보고서 (2021.12),5. 재무제표 주석,OK,NaN,0.0,0.0,OK
